# Data visualization with HoloViews

Load libraries, and do some configuration.

In [ ]:
import math
import pandas as pd
import holoviews as hv
import numpy as np
import scipy.stats as stats
%load_ext holoviews.ipython

## Reading and plotting the data

Read the data file.

In [ ]:
data = pd.read_csv('data.csv')
data

For convenience, get a list of the column names containing the genes.

In [ ]:
genes = list(data.columns[3:])
genes

In [ ]:
agents = list(data.columns[1:3])
agents

In [ ]:
%output size=100

In [ ]:
%%output size=250
hv.DFrame(data).scatter('IPO8', 'YWHAZ')

This seems odd, checking, it turns out that the samples have been processed in two batches, the second starting from ID 601.  Create two plots, one with only the data of the first batch, the other with that of the second batch, and create an overlay.  Create a mask to select either batch 1 or 2.

## Overlays

In [ ]:
batch1 = data['ID'] < 601
batch2 = data['ID'] >= 601

In [ ]:
%%output size=250
batch1_plot = hv.DFrame(data[batch1]).scatter('IPO8', 'YWHAZ', label='batch 1')
batch2_plot = hv.DFrame(data[batch2]).scatter('IPO8', 'YWHAZ', label='batch 2')
batch1_plot * batch2_plot

The plot above visually proves that there the batches are indeed the cause of the two clusters observed in the data.

## Linear regression

In [ ]:
slope_1, intercept_1, r_1, _, stderr_1 = stats.linregress(data.loc[batch1, 'IPO8'], data.loc[batch1, 'YWHAZ'])
slope_2, intercept_2, r_2, _, stderr_2 = stats.linregress(data.loc[batch2, 'IPO8'], data.loc[batch2, 'YWHAZ'])

In [ ]:
x = np.linspace(24.0, 29.0, 11)
y_1 = slope_1*x + intercept_1
y_2 = slope_2*x + intercept_2

In [ ]:
fit1_plot = hv.Curve(zip(x, y_1), label='batch 1')
fit2_plot = hv.Curve(zip(x, y_2), label='batch 2')

In [ ]:
%%output size=250
batch1_plot * fit1_plot * batch2_plot * fit2_plot

## Plotting distributions

To visualize differences in technical variation on the threatment of the samples, plot the distribution of the deviations from the regression.

In [ ]:
def fit_func1(x):
    return slope_1*x + intercept_1
def fit_func2(x):
    return slope_2*x + intercept_2

In [ ]:
delta_1 = (data.loc[batch1, 'YWHAZ'] - fit_func1(data.loc[batch1, 'IPO8']))*np.sin(np.arctan(slope_1))
delta_2 = (data.loc[batch2, 'YWHAZ'] - fit_func2(data.loc[batch2, 'IPO8']))*np.sin(np.arctan(slope_2))

In [ ]:
%%output size=250
delta_plot1 = hv.Distribution(delta_1, label='batch 1')
delta_plot2 = hv.Distribution(delta_2, label='batch 2')
delta_plot1 * delta_plot2

## GridSpace

Show the gene expression versus the agent concentrations.

In [ ]:
keys = [(g, a) for g in genes for a in agents]

In [ ]:
plots = {k: hv.DFrame(data).scatter(*k) for k in keys}

In [ ]:
%%output size=400
hv.GridSpace(plots, kdims=['gene', 'agent'])

## HoloMap

In [ ]:
%%output size=250
hv.HoloMap(plots, kdims=['gene', 'agent'])